# 3. Análisis transformer
Para el segundo chatbot utilice un transformer (por ejemplo BERT)

## Primer punto
Utilice el modelo transformer para clasificar el texto de entrada, y para
extraer la respuesta de la tabla de preguntas y respuestas cuando el
mensaje sea del tipo “información”.

In [1]:
# Librería para manipular la data
import pandas as pd
# Librería para utilizar transformers
# Las "pipeline" permite utilizar multiples capas de transformers
# pueden tomar predefinidas arquitecturas en huggingface
# https://huggingface.co/docs/transformers/main_classes/pipelines
from transformers import pipeline
# Importo torch que se utilizara para entrenar el modelo
import torch
# Clase para cargar el tokenizador
from transformers import AutoTokenizer
# Clase para secuencias a clasificación (N -> 1)
from transformers import AutoModelForSequenceClassification

# clases para entrenamiento
from transformers import Trainer, TrainingArguments

# enumeraciones
from enum import Enum

# Para ignorar warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
QA_PATH = "../data/FAQ/tablaQA.xls"
RESPUESTAS_DEFECTO_PATH = "../data/FAQ/respuestasDefecto.xls"
TIPOS_MENSAJES_PATH = "../data/FAQ/tiposmensajes.xlsx"
TIPOS_MENSAJES_TEST_PATH = "../data/FAQ/tiposmensajes_test.xlsx"
# Cargo un modelo preentrenado en español
#MODEL_NAME='mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es'
MODEL_NAME="mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
class TipoMensaje(Enum):
    Saludo = "saludo"
    Despedida = "despedida"
    Nombre = "nombre"
    Informacion = "informacion"
    def __str__(self) -> str:
        return self.value
    def __eq__(self, __o: str) -> bool:
        return __o == self.value

In [3]:
# cargo QA dataframe
df_qa = pd.read_excel(QA_PATH)
df_qa['Preguntas'] = df_qa.Preguntas.map(str.strip)
# cargo Respuestas dataframe
df_respuestas = pd.read_excel(RESPUESTAS_DEFECTO_PATH)
# cargo Tipo Mensajes dataframe
df_tipo_mensajes = pd.read_excel(TIPOS_MENSAJES_PATH)
df_tipo_mensajes_test = pd.read_excel(TIPOS_MENSAJES_TEST_PATH)

In [4]:
df_qa.head()

,Num,Preguntas,Respuestas
0,1,¿Qué es la Encuesta Casen?,La Encuesta de Caracterización Socioeconómica ...
1,2,¿Qué instituciones participan en la realizació...,Las instituciones que participan en la realiza...
2,3,¿Cada cuánto tiempo se realiza la Encuesta Casen?,La Encuesta Casen es realizada regularmente po...
3,4,¿Cuál es el tamaño de la muestra de la Encuest...,El tamaño de la muestra y su nivel de precisió...
4,5,¿Se puede acceder a las bases de datos de la E...,Sí. Las bases de datos innominadas de la Encue...


Se genera un modelo para clasificar el texto en:
1. Saludo
1. Despedida
1. Nombre
1. Información

Se crea un dataset para el entrenamiento

In [5]:
LABELS = [TipoMensaje.Saludo, TipoMensaje.Despedida, TipoMensaje.Nombre, TipoMensaje.Informacion]
LABELS = list(map(str, LABELS))
df_tipo_mensajes["label"] = df_tipo_mensajes.Tipo.map(lambda tipo: [tipo == label for label in LABELS])
df_tipo_mensajes_test["label"] = df_tipo_mensajes_test.Tipo.map(lambda tipo: [tipo == label for label in LABELS])
# Creo el dataset, los minimos elementos a implementar son
# __init__, __len__ y __getitem__
# esto es porque itera con un for simple
class PandasDataset(torch.utils.data.Dataset):
  def __init__(self, df, x, y, tokenizer):    
    self.x = df[x]
    self.y = df[y]
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.x)

  def __getitem__(self, ix):
    return {
                **self.tokenizer(self.x[ix], truncation=True, padding="max_length", max_length=50),
                **{"label": self.y[ix], "text": self.x[ix]}
            }

Se crea el modelo a entrenar

In [6]:
%%capture
# Creo el modelo para clasificación, para el numero de label de nuestro problema
# model_name es el nombre del modelo que quiero usar como base
device = "cuda" if torch.cuda.is_available() else "cpu"
NUM_LABELS = len(LABELS)
model = (AutoModelForSequenceClassification
         .from_pretrained(MODEL_NAME, num_labels=NUM_LABELS, id2label=LABELS)
         .to(device))
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


Some weights of the model checkpoint at mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForSequenceClassification: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to b

Se configura los parametros de entrenamientos

In [7]:
# Determino los parametros del entrenamiento
BATCH_SIZE = 2
TRAIN_EPOCHS=10
logging_steps = len(df_qa) // BATCH_SIZE
training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=TRAIN_EPOCHS,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=BATCH_SIZE,
                                  per_device_eval_batch_size=BATCH_SIZE,
                                  load_best_model_at_end=True,
                                  #metric_for_best_model="f1",
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,)

Se entrena el modelo seleccionado.

*Obs.* Por la cantidad baja de ejemplos, se útiliza todos los registros para el entrenamiento, pero en un caso real, debería separarse un subset para validación.

In [8]:
%%capture
training_dateset = PandasDataset(df_tipo_mensajes, "Mensaje", "label", tokenizer)
test_dateset = PandasDataset(df_tipo_mensajes_test, "Mensaje", "label", tokenizer)
trainer = Trainer(model=model, args=training_args, train_dataset=training_dateset, eval_dataset=test_dateset)
trainer.train();

***** Running training *****
  Num examples = 115
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 580
***** Running Evaluation *****
  Num examples = 44
  Batch size = 2
Saving model checkpoint to results/checkpoint-58
Configuration saved in results/checkpoint-58/config.json
Model weights saved in results/checkpoint-58/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 44
  Batch size = 2
Saving model checkpoint to results/checkpoint-116
Configuration saved in results/checkpoint-116/config.json
Model weights saved in results/checkpoint-116/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 44
  Batch size = 2
Saving model checkpoint to results/checkpoint-174
Configuration saved in results/checkpoint-174/config.json
Model weights saved in results/checkpoint-174/pytorch_model.bin
***** Running Evaluation *****
  Nu

In [9]:
df_qa['Preguntas'] = df_qa.Preguntas.map(str.strip)
# cargo Respuestas dataframe
df_respuestas = pd.read_excel(RESPUESTAS_DEFECTO_PATH)
# cargo Tipo Mensajes dataframe
df_tipo_mensajes = pd.read_excel(TIPOS_MENSAJES_PATH)

Se crea el bot para responder según el tipo de entrada

In [10]:
class Bot():
    def __init__(self, pipe_qa, context, pipe_classifier, df_respuestas) -> None:
        self.pipe_qa = pipe_qa
        self.context = context
        self.classifier = pipe_classifier
        self.df_respuestas = df_respuestas
        self.nombre = None
        self.chat = []
    def __call__(self, text) -> str:
        # Guardo el texto en la lista
        self.chat.append(f"Usuario: {text}")
        # determino la clase de entrada
        clase = self.classifier(text)[0]["label"]
        
        if clase == TipoMensaje.Saludo:
            self.saludar(text)
        elif clase == TipoMensaje.Despedida:
            self.despedirse(text)
        elif clase == TipoMensaje.Nombre:
            self.guardarNombre(text)
        elif clase == TipoMensaje.Informacion:
            self.responder(text)
        return "\n".join(self.chat)
    def replyBot(self, text):
        self.chat.append(f"Bot: {text}")
    def saludar(self, text):
        saludo = self.df_respuestas[self.df_respuestas.tipo == TipoMensaje.Saludo].sample(1).respuesta.iloc[0]
        if self.nombre is None:
            saludo = f"{saludo}, ¿cual es tu nombre?"
        self.replyBot(saludo)
    def despedirse(self, text):
        self.replyBot(
            self.df_respuestas[self.df_respuestas.tipo == TipoMensaje.Despedida].sample(1).respuesta.iloc[0]
            )
    def responder(self, text):
        self.replyBot(
            self.pipe_qa(question=text, context=self.context)["answer"]
        )
    def guardarNombre(self, text):
        self.nombre = text
        self.replyBot(f"mucho gusto {self.nombre}")
        self.replyBot("¿en que podemos ayudarte?")
    def __repr__(self) -> str:
        return "\n".join(self.chat)

In [11]:
%%capture
pipe_qa = pipeline("question-answering", model=MODEL_NAME)
context = "\n".join(df_qa.Respuestas.values)
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0 if device=="cuda" else -1)

loading configuration file https://huggingface.co/mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es/resolve/main/config.json from cache at /home/gerardo/.cache/huggingface/transformers/17330f67d8c327c0b1699be552404022f63be5db79858b26484fc847da416eb9.2e4532ea7d3ba93d791168876c978107ea0cba47d2b0736de7c9139e9670eff4
Model config BertConfig {
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}

loading configuration file https://huggingface.co

## Segundo punto
Reporte el tipo de red, y las métricas de entrenamiento usadas. Si requiere generar más ejemplos puede generar variaciones de sus tablas
usando el tutorial del www.github.com/makcedward/nlpaug/blob/
master/example/textual\_augmenter.ipynb

Para ambos modelos se utilizó `BertForQuestionAnswering` con los pesos `mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es`, pero con las herramientas de huggingface se modifica el modelo para utilizarlo para clasificación.

Esto permite la transformación a `BertForSequenceClassification` y ajustarlo a un clasificador de 4 clases con
```
(classifier): Linear(in_features=768, out_features=4, bias=True)
```

In [26]:
def clasificador(text):
    return classifier(text)[0]['label']

df_tipo_mensajes_test["y_pred"] = df_tipo_mensajes_test.Mensaje.map(clasificador)
y_true = df_tipo_mensajes_test["Tipo"].map(LABELS.index).values
y_pred = df_tipo_mensajes_test["y_pred"].map(LABELS.index).values


In [37]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(y_true, y_pred))
confusion_matrix(y_true, y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00        30
           3       1.00      1.00      1.00         6

    accuracy                           1.00        44
   macro avg       1.00      1.00      1.00        44
weighted avg       1.00      1.00      1.00        44



array([[ 5,  0,  0,  0],
       [ 0,  3,  0,  0],
       [ 0,  0, 30,  0],
       [ 0,  0,  0,  6]])

## Tercer Punto
Reporte el resultado con los textos de prueba.

Se prueba pregunta respuesta pipeline

In [12]:
pipe_qa(question="¿Que es la CASEN?", context=context)

{'score': 0.6107734441757202,
 'start': 5231,
 'end': 5282,
 'answer': 'Encuesta de Caracterización Socioeconómica Nacional'}

Se prueba en conjunto el robot

In [13]:
bot = Bot(pipe_qa, context, classifier, df_respuestas)
bot("Hola")
bot("Gerardo Villarroel")
bot("me gustaría saber que es la encuesta CASEN")
bot("que es la CASEN?")
print(bot("chao"))

Usuario: Hola
Bot: Buenos días, ¿cual es tu nombre?
Usuario: Gerardo Villarroel
Bot: mucho gusto Gerardo Villarroel
Bot: ¿en que podemos ayudarte?
Usuario: me gustaría saber que es la encuesta CASEN
Bot: un instrumento aplicado a una muestra aleatoria y anónima de hogares
Usuario: que es la CASEN?
Bot: Encuesta de Caracterización Socioeconómica Nacional
Usuario: chao
Bot: Adiós!
